In [ ]:
####################################################################################################################
# Competition   : Dry Beans Classification
# Team Members  : 1- Abdullah Abdelhakeem
#                 2- Mohamed Sebaie                   
#                 3- Mohamed Moustafa
#                 4- Ossama Ahmed
#                 5- Mahmoud Osama
#                 6- Hazem
#
# Problem :       Supervised Classification
# version :       0.0.1
#
###################################################################################################################

# Import Required Libraries

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import statsmodels.api as sm
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import ConfusionMatrixDisplay,confusion_matrix
from mlxtend.plotting import plot_confusion_matrix
#####################################################################
# for visualizations
from pandas import plotting
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
plt.style.use('fivethirtyeight')
sns.set(rc={'figure.figsize': [7, 7]}, font_scale=1.2)
###########################################################
%config Completer.use_jedi=False
pd.set_option("display.max_columns", None)

# ignore the warnings
import warnings
warnings.filterwarnings("ignore")
np.set_printoptions(suppress=True)

# for path
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Read Train and Test Data

In [3]:
dfTrainO=pd.read_csv("train.csv")
print('---'*30)
print('Train Head')
print('---'*30)
display(dfTrainO.head())
dfTestO=pd.read_csv("test.csv")
print('---'*30)
print('Test Head')
print('---'*30)
display(dfTestO.head())

------------------------------------------------------------------------------------------
Train Head
------------------------------------------------------------------------------------------


,ID,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,y
0,0,58238,971.303,397.202654,186.945510,2.124697,0.882317,58977,272.306674,0.604756,0.987470,0.775723,0.685561,0.006820,0.000929,0.469994,0.998595,HOROZ
1,1,44515,757.601,265.590470,213.967453,1.241266,0.592420,44780,238.071960,0.771745,0.994082,0.974620,0.896387,0.005966,0.002376,0.803510,0.997370,SEKER
2,2,31662,653.783,239.902428,168.421505,1.424417,0.712136,32034,200.781748,0.801407,0.988387,0.930853,0.836931,0.007577,0.002293,0.700453,0.997737,DERMASON
3,3,38778,734.020,268.446281,184.061923,1.458456,0.727925,39208,222.201897,0.766137,0.989033,0.904439,0.827733,0.006923,0.002005,0.685142,0.999250,DERMASON
4,4,42530,775.403,281.212824,193.236878,1.455275,0.726511,43028,232.703412,0.729816,0.988426,0.888895,0.827499,0.006612,0.001912,0.684755,0.996507,SIRA


------------------------------------------------------------------------------------------
Test Head
------------------------------------------------------------------------------------------


,ID,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4
0,10834,57659,955.434,387.757607,196.625782,1.972059,0.861896,60188,270.949661,0.620790,0.957982,0.793735,0.698760,0.006725,0.000989,0.488266,0.962889
1,10835,28772,630.362,229.990785,159.609367,1.440960,0.719993,29127,191.399185,0.767458,0.987812,0.909913,0.832204,0.007994,0.002365,0.692563,0.997956
2,10836,54677,911.022,308.853903,226.398571,1.364204,0.680198,55858,263.850182,0.753013,0.978857,0.827860,0.854288,0.005649,0.001856,0.729808,0.995607
3,10837,24827,578.304,214.192699,147.788172,1.449322,0.723831,25121,177.794033,0.716508,0.988297,0.932871,0.830066,0.008627,0.002526,0.689009,0.998596
4,10838,141850,1432.847,524.923471,346.910920,1.513136,0.750492,143680,424.981211,0.804617,0.987263,0.868240,0.809606,0.003701,0.000981,0.655462,0.991803


__________________________________________________

In [6]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(dfTrainO, test_size=0.33, random_state=0,stratify =dfTrainO['y'])
X_train = train_df.drop(columns=["ID","y"])
y_train = train_df['y']
################################################################################################
X_val = val_df.drop(columns=["ID","y"])
y_val = val_df['y']

In [7]:
def normalize(df):
    result = df.copy()
    for feature_name in df.columns:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result

# X_train=normalize(X_train)
# X_val=normalize(X_val)
# dfTest=normalize(dfTest)

# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# scaler.fit(X_train)
# X_train = scaler.transform(X_train)
# X_val = scaler.transform(X_val)

# from sklearn.preprocessing import minmax_scale
# X_train = minmax_scale(X_train, feature_range=(0, 1))
# X_val = minmax_scale(X_val, feature_range=(0, 1))

# from sklearn.preprocessing import RobustScaler
# rb = RobustScaler()
# X_train= rb.fit_transform(X_train)
# X_val = rb.fit_transform(X_val)

from sklearn.preprocessing import StandardScaler
scaler  = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)

In [9]:
from imblearn.over_sampling import SMOTE
smt = SMOTE(random_state=0)
X_train, y_train = smt.fit_resample(X_train, y_train)
np.unique(y_train , return_counts=True)

(array(['BARBUNYA', 'BOMBAY', 'CALI', 'DERMASON', 'HOROZ', 'SEKER', 'SIRA'],
       dtype=object),
 array([1900, 1900, 1900, 1900, 1900, 1900, 1900], dtype=int64))

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier,GradientBoostingClassifier,AdaBoostClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

In [11]:
models = {
    # "LogisticRegression":           LogisticRegression(),
    "KNeighborsClassifier":         KNeighborsClassifier(), 
    "DecisionTreeClassifier":       DecisionTreeClassifier(),
    "SupportVectorMachine":         SVC(C=1.0,kernel='rbf',gamma='auto'),
    "LinearDiscriminantAnalysis":   LinearDiscriminantAnalysis(),
    "GaussianNB":                   GaussianNB(),
    "SGDClassifier":                SGDClassifier(),
    "RandomForestClassifier":       RandomForestClassifier(),
    "BaggingClassifier":            BaggingClassifier(),
    "CatBoostClassifier":           CatBoostClassifier(verbose=False,loss_function='MultiClass'),
    "LGBMClassifier":               LGBMClassifier(),
    "GradientBoostingClassifier":   GradientBoostingClassifier(),
    "XGBClassifier":                XGBClassifier(eval_metric='mlogloss')
}

In [12]:
X_train.shape

(13300, 16)

In [13]:
y_train.shape

(13300,)

In [14]:
for name, model in models.items():
    print(f'Using model: {name}')
    print('-'*30)
    model.fit(X_train, y_train)
    y_trainhat = model.predict(X_train)
    y_valhat = model.predict(X_val)
    acc = accuracy_score(y_val, y_valhat)
    # print(f'Train_Report: {classification_report(y_train,y_trainhat)}')
    print(f'Train_Accuracy: {accuracy_score(y_train,y_trainhat)}')
    # print(f'Validation_Report: {classification_report(y_val,y_valhat)}')
    print(f'Validation_Accuracy: {accuracy_score(y_val,y_valhat)}')
    print('**'*30)

Using model: KNeighborsClassifier
------------------------------
Train_Accuracy: 0.9677443609022557
Validation_Accuracy: 0.9147091722595079
************************************************************
Using model: DecisionTreeClassifier
------------------------------
Train_Accuracy: 1.0
Validation_Accuracy: 0.8831096196868009
************************************************************
Using model: SupportVectorMachine
------------------------------
Train_Accuracy: 0.9479699248120301
Validation_Accuracy: 0.927013422818792
************************************************************
Using model: LinearDiscriminantAnalysis
------------------------------
Train_Accuracy: 0.9196240601503759
Validation_Accuracy: 0.8945749440715883
************************************************************
Using model: GaussianNB
------------------------------
Train_Accuracy: 0.9085714285714286
Validation_Accuracy: 0.8956935123042505
************************************************************
Using model: 

In [ ]:
print("Number of mislabeled points out of a total %d points : %d"% (X_test.shape[0], (y_test != y_pred).sum()))

## Test Data

In [ ]:
# ############################################
# TestYTrue= pd.read_csv("yTest.csv")
# yTrueTest=TestYTrue["y"]
# ############################################

ID=dfTestO["ID"]


In [ ]:
# dfTestNEW.loc[: , dfTestNEW.columns!="ID"]
dfTestO

In [ ]:
# X_test = dfTestNEW.loc[: , dfTestNEW.columns!="ID"]


In [ ]:
Model=CatBoostClassifier(iterations=700, learning_rate=0.2,verbose=False)
Model.fit(X_train, y_train)

y_trainhat = Model.predict(X_train)
y_valhat = Model.predict(X_val)
acc = accuracy_score(y_val, y_valhat)
# print(f'Train_Report: {classification_report(y_train,y_trainhat)}')
print(f'Train_Accuracy: {accuracy_score(y_train,y_trainhat)}')
# print(f'Validation_Report: {classification_report(y_val,y_valhat)}')
print(f'Validation_Accuracy: {accuracy_score(y_val,y_valhat)}')
print('**'*30)

In [ ]:
y_test_predicted = Model.predict(X_test)

dfTestO['y'] = y_test_predicted

dfTestO.drop(columns="ID",inplace=True)
dfTest=pd.concat([dfTestO, ID],axis=1)

#dfTestEncoded2[['ID', 'y']].to_csv('/kaggle/working/submission.csv', index=False)
dfTest[['ID', 'y']].to_csv('submission.csv', index=False)

In [ ]:
dfTest[['ID', 'y']]

# Test ActualData vs TestData

In [22]:
#DifferenceActualTest.csv
testAcual = pd.read_csv("update.csv" , )

In [23]:
testAcual.columns

Index(['ID', 'Area', 'Perimeter', 'MajorAxisLength', 'MinorAxisLength',
       'AspectRation', 'Eccentricity', 'ConvexArea', 'EquivDiameter', 'Extent',
       'Solidity', 'roundness', 'Compactness', 'ShapeFactor1', 'ShapeFactor2',
       'ShapeFactor3', 'ShapeFactor4', 'Class'],
      dtype='object')

In [24]:
from imblearn.over_sampling import SMOTE
smt = SMOTE(random_state=0)
X_train, y_train = smt.fit_resample(X_train, y_train)
np.unique(y_train , return_counts=True)

(array(['BARBUNYA', 'BOMBAY', 'CALI', 'DERMASON', 'HOROZ', 'SEKER', 'SIRA'],
       dtype=object),
 array([1900, 1900, 1900, 1900, 1900, 1900, 1900], dtype=int64))

In [25]:
testAcual["Class"]

0       SEKER
1       SEKER
2       SEKER
3       SEKER
4       SEKER
        ...  
2716      NaN
2717      NaN
2718      NaN
2719      NaN
2720      NaN
Name: Class, Length: 2721, dtype: object